# Домашка 3

1. обучить несколько разных моделей на наборе данных ССЗ (train_case2.csv): логрег, бустинг, лес и т.д - на ваш выбор 2-3 варианта
2. при обучении моделей обязательно использовать кроссвалидацию
3. вывести сравнение полученных моделей по основным метрикам классификации: pr/rec/auc/f_score (можно в виде таблицы, где строки - модели, а столбцы - метрики)
4. сделать выводы о том, какая модель справилась с задачей лучше других
5. (опциональный вопрос) какая метрика (precision_recall_curve или roc_auc_curve) больше подходит в случае сильного дисбаланса классов? (когда объектов одного из классов намного больше чем другого). 

p.s.В вопросе проще разобраться, если вспомнить оси на графике roc auc curve и рассмотреть такой пример:

Имеется 100000 объектов, из которых только 100 - класс "1" (99900 - класс "0", соответственно). 
Допустим, у нас две модели:

- первая помечает 100 объектов как класс 1, но TP = 90
- вторая помечает 1000 объектов как класс 1, но TP такой же - 90

Какая модель лучше и почему? И что позволяет легче сделать вывод - roc_auc_curve или precision_recall_curve?

In [18]:
import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.model_selection import cross_val_score, train_test_split
from scipy.sparse import hstack
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.metrics import precision_recall_curve, roc_curve, roc_auc_score, confusion_matrix

In [5]:
df = pd.read_csv('train_case2.csv', ';')
print(df.shape)
df.head(3)

(70000, 13)


,id,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
0,0,18393,2,168,62.0,110,80,1,1,0,0,1,0
1,1,20228,1,156,85.0,140,90,3,1,0,0,1,1
2,2,18857,1,165,64.0,130,70,3,1,0,0,0,1


In [8]:
#разделим данные на train/test
X_train, X_test, y_train, y_test = train_test_split(df.drop('cardio', 1), 
                                                    df['cardio'], random_state=0)

X_train.shape, y_train.shape, X_test.shape, y_test.shape

((52500, 12), (52500,), (17500, 12), (17500,))

In [9]:
class ColumnSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[self.key]
    
class NumberSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    Use on numeric columns in the data
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[[self.key]]
    
class OHEEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key
        self.columns = []

    def fit(self, X, y=None):
        self.columns = [col for col in pd.get_dummies(X, prefix=self.key).columns]
        return self

    def transform(self, X):
        X = pd.get_dummies(X, prefix=self.key)
        test_columns = [col for col in X.columns]
        for col_ in test_columns:
            if col_ not in self.columns:
                X[col_] = 0
        return X[self.columns]


from sklearn.preprocessing import StandardScaler


continuos_cols = ['age', 'height', 'weight', 'ap_hi', 'ap_lo']
cat_cols = ['gender', 'cholesterol']
base_cols = ['gluc', 'smoke', 'alco', 'active']

continuos_transformers = []
cat_transformers = []
base_transformers = []

for cont_col in continuos_cols:
    transfomer =  Pipeline([
                ('selector', NumberSelector(key=cont_col)),
                ('standard', StandardScaler())
            ])
    continuos_transformers.append((cont_col, transfomer))
    
for cat_col in cat_cols:
    cat_transformer = Pipeline([
                ('selector', ColumnSelector(key=cat_col)),
                ('ohe', OHEEncoder(key=cat_col))
            ])
    cat_transformers.append((cat_col, cat_transformer))
    
for base_col in base_cols:
    base_transformer = Pipeline([
                ('selector', NumberSelector(key=base_col))
            ])
    base_transformers.append((base_col, base_transformer))

In [10]:
from sklearn.pipeline import FeatureUnion

feats = FeatureUnion(continuos_transformers+cat_transformers+base_transformers)
feature_processing = Pipeline([('feats', feats)])

feature_processing.fit_transform(X_train)

array([[-1.73391771,  0.6873301 ,  0.74843904, ...,  1.        ,
         0.        ,  1.        ],
       [-1.67343538,  0.07758923, -0.29640123, ...,  0.        ,
         0.        ,  1.        ],
       [ 0.13738132,  1.17512278, -0.15708919, ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [ 1.17775864,  1.17512278, -0.15708919, ...,  0.        ,
         0.        ,  1.        ],
       [-0.47190715, -1.38578883,  0.74843904, ...,  0.        ,
         0.        ,  1.        ],
       [ 0.38174619,  0.56538192, -0.08743318, ...,  0.        ,
         0.        ,  1.        ]])

In [102]:
model_types = {'Логистичиская регрессия': LogisticRegression(random_state = 42),
               'Градиентный бустинг': GradientBoostingClassifier(random_state = 42),
               'Случайный лес': RandomForestClassifier(random_state = 42),
              }

models = []
for key in model_types:
    
    classifier = Pipeline([
        ('features',feats),
        ('classifier', model_types[key]),
    ])
    
    model = {'title': key, 'classifier': classifier}
    models.append(model)

In [103]:
%%time
#обучим наши модели
for model in models:
    
    classifier = model['classifier']
    
    #запустим кросс-валидацию
    cv_scores = cross_val_score(classifier, X_train, y_train, cv=16, scoring='roc_auc')
    cv_score = np.mean(cv_scores)
    cv_score_std = np.std(cv_scores)
    print('Model: {}.\tCV score is {}+-{}'.format(model['title'], cv_score, cv_score_std))

    #обучим пайплайн на всем тренировочном датасете
    classifier.fit(X_train, y_train)
    model['y_score'] = classifier.predict_proba(X_test)[:, 1]

Model: Логистичиская регрессия.	CV score is 0.7867401104915408+-0.00852135511666111
Model: Градиентный бустинг.	CV score is 0.8025124517417064+-0.007075372179995901
Model: Случайный лес.	CV score is 0.7734501681056019+-0.007171140345435727
Wall time: 2min 48s


In [105]:
for model in models:    
    b=1
    precision, recall, thresholds = precision_recall_curve(y_test.values, model['y_score'])
    fscore = (1+b**2)*(precision * recall) / (b**2*precision + recall)
    # locate the index of the largest f score
    ix = np.argmax(fscore)
    model['precision'] = precision[ix] 
    model['recall'] = recall[ix]
    model['fscore'] = fscore[ix]
    
    print('Best Threshold=%f, F-Score=%.3f, Precision=%.3f, Recall=%.3f. Model: %s' % (thresholds[ix], 
                                                                        fscore[ix],
                                                                        precision[ix],
                                                                        recall[ix],
                                                                        model['title']))

Best Threshold=0.386937, F-Score=0.730, Precision=0.647, Recall=0.838. Model: Логистичиская регрессия
Best Threshold=0.394947, F-Score=0.740, Precision=0.698, Recall=0.788. Model: Градиентный бустинг
Best Threshold=0.350000, F-Score=0.719, Precision=0.643, Recall=0.816. Model: Случайный лес


In [108]:
for model in models:
    
    #странный косяк поймал, если не переинецилизировать каждый раз, выдает ошибку
    from sklearn.metrics import roc_auc_score 
    
    roc_auc_score = roc_auc_score(y_true=y_test, y_score=model['y_score'])
    model['roc_auc_score'] = roc_auc_score   
    print("roc auc score: {}. Model: {} ".format(roc_auc_score, model['title']))

roc auc score: 0.7840347790421852. Model: Логистичиская регрессия 
roc auc score: 0.8026153641179974. Model: Градиентный бустинг 
roc auc score: 0.7710366181802983. Model: Случайный лес 


In [115]:
# формулируем результирующую таблицу
titles = []
precisions = []
recalls = []
fscores = []
roc_aucs = []

for model in models:
    titles.append(model['title'])
    precisions.append(model['precision'])
    recalls.append(model['recall'])
    fscores.append(model['fscore'])
    roc_aucs.append(model['roc_auc_score'])
    
result = pd.DataFrame({'precision': precisions, 'recall': recalls, 'fscore': fscores, 'roc auc score': roc_aucs}, 
                          index=titles)
result

,precision,recall,fscore,roc auc score
Логистичиская регрессия,0.647431,0.837558,0.730323,0.784035
Градиентный бустинг,0.697848,0.788134,0.740248,0.802615
Случайный лес,0.642669,0.815553,0.718863,0.771037


## Ответим на вопросы

__4. сделать выводы о том, какая модель справилась с задачей лучше других__

Нельзя сказать однозначно, какая модель справилась лучше. Хотя интегральная оценка `roc_auc_score`, рейтинг по оценкам `precision` и `recall`.  Точность получилась выше у градиентного бустинга выше, но полнота выше у логистической модели и случайного леса. Что объясняется склонность бустинга к переобучению. Логистическая модель наооборот, может показать больший сдвиг, но меньший разброс (более устойчивая). 

Лес же оказался посередине. С одной стороны по точности он почти равен логистической регрессии, но полнота у него существенно выше.

Какая модель лучше справилась с задачей будет определяться задачей. Если нам нужна максимизация точности при заданном уровне полноты, стоит выбрать бустинг. Если максимальная полноста, то возможно лучше подойдет регрессия. Если нужна высокая обучения (логистическая регрессия обучилась в 3 раза быстрее конкурентов) при заданном уровне метрик качества, возможно регрессия подойдет лучше.




__5. (опциональный вопрос) какая метрика (precision_recall_curve или roc_auc_curve) больше подходит в случае сильного дисбаланса классов? (когда объектов одного из классов намного больше чем другого)__

Я думаю, что при сильном дисбалансе классов лучше подойдет `precision_recall_curve`. Потому что когда одного класса существенно меньше, чем другого, визуальное отображение кривой `roc_auc_curve` может приводить к неточным выводам. Количество шагов по вертикали будет намного меньше количества шагов по горизонтали. И кривая будет выглядеть не одинаково при отображении значений одного класса и другого класса. 

`precision_recall_curve` наоборот при любом дисбалансе классов будет выглядеть одинаково. И как графически, так и аналитически можно всегда точно определить удобный порог в зависимости от решаемой задачи и увидеть насколько сильно будут отличаться график в области выбранного порога если немного сместиться влево или вправо.